In [3]:
%matplotlib inline
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
import surprise
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Ratings that have to be predicted (items and users specified in sample_submission file)

In [4]:
from data_helpers import read_csv_sample

INPUT_PATH = "../data/sample_submission.csv"
ids = read_csv_sample(INPUT_PATH)

# Co-clustering

In [6]:
from surprise.model_selection import GridSearchCV
from surprise import CoClustering
from surprise import Dataset
from surprise import Reader

# path to dataset file
file_path = "../data/data_surprise.csv"
reader = Reader(line_format='user item rating', sep=',', skip_lines=1)
data = Dataset.load_from_file(file_path, reader=reader)
trainset = data.build_full_trainset()

param_grid = {'n_cltr_u': [1], 'n_cltr_i': [1],
              'n_epochs': [20]}

gs = GridSearchCV(CoClustering, param_grid, measures=['rmse', 'mae'], cv=2)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.0097189924111927
{'n_cltr_u': 1, 'n_cltr_i': 1, 'n_epochs': 20}


In [8]:
algo = CoClustering(n_cltr_u=1, n_cltr_i=1, n_epochs=100)
algo.fit(trainset)

In [10]:
predictions = []

for i in range(len(ids[0])):
    pred = algo.predict(str(ids[0][i]), str(ids[1][i]))
    predictions.append(round(pred.est))
    
print(len(predictions))

1176952


### Save output for submission

In [11]:
from data_helpers import create_csv_submission

OUTPUT_PATH = "../data/submission.csv"
create_csv_submission(ids, predictions, OUTPUT_PATH)
print("File submission.csv ready to be submitted !")

File submission.csv ready to be submitted !
